<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

## Bootstrap with a Cowbow Hat Demonstration

* in class I bring in 3 red blocks, 2 yellow blocks and my cowboy hat, yes I have one, recall I was a farmhand in Northern Alberta, Canada. Aside, ever heard me say, 'Howdy'?

* then I have 3 students volunteer, one holds the hat, one draws balls with replacement and one records the results on the board

* through multiple bootstrap realizations we demonstrate the use of bootstrap to calculate uncertainty in the proportion from the sample itself

* with this workflow we all provide an interactive plot demonstration with matplotlib and ipywidget packages

#### Michael Pyrcz, Professor, The University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1) | [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy)

#### Bootstrap

Uncertainty in sample statistics:
* one source of uncertainty is the paucity of data.
* do 200 or even less samples provide a precise (and accurate estimate) of the mean? standard deviation? skew? P13?

Would it be useful to know the uncertainty in these statistics due to the limited number of samples?
* what is the impact of uncertainty in the mean porosity, e.g., 20%+/-2%?

**Bootstrap** is a method to assess the uncertainty in a sample statistic by repeated random sampling with replacement, new realizations of the data.

#### Bootstrap Assumptions and Limitations

Assumptions:
* sufficient, representative sampling, identical (stationarity over the population), idependent samples (no correlation between the samples)

Limitations:
1. assumes the samples are representative, does not correct for model bias 
2. assumes stationarity, identically distributed data over the area of interest
3. only accounts for uncertainty due to too few samples, e.g., no uncertainty due to changes away from data
4. does not account for boundary of area of interest 
5. assumes the samples are independent, no correlations over time or space
6. does not account for other local information sources, no local model conditioning

#### The Empirical Bootstrap Approach (Efron, 1982)

Statistical resampling procedure to calculate accuracy in a sample estimate, i.e., uncertainty in a calculated statistic, from the data itself.

* **mimicks** the sampling process, to proceed with multiple realizations of the data sample

* **one of resampling methods** like permutation tests for hypothesis testing and cross validation for testing predictive model performance

* **inferential**, approximates the distribution of uncertainty with the empirical distribution function, realizations of the statistic of interest 

* **flexible**, may be applied to calculate the uncertainty in any statistic

* **machine learning** applies bootstrap to build ensembles of models for ensemble estimate aggragation to reduce model variance and improve model accuracy, this is known as model bagging

* **spatial bootstrap** is an advanced form that accounts for spatial correlation, conditioning and nonstationarity (Journel, 1993). This is based on general model resampling methods. See my demonstrations:

* [PythonDataBasics_SpatialBootstrap](https://github.com/GeostatsGuy/PythonNumericalDemos/blob/master/PythonDataBasics_SpatialBootstrap.ipynb)

* [Spatial_Bootstrap](https://github.com/GeostatsGuy/PythonNumericalDemos/blob/master/Spatial_Bootstrap.ipynb) where I demonstrate the LU simulation-based spatial bootstrap method (Deutsch, 2004).

#### Bootstrap Workflow

Here's the general steps of bootstrap:

1. assemble a sample set, must be representative, and reasonable to assume independence between samples

2. optional: build a cumulative distribution function (CDF)
    * may account for declustering weights, tail extrapolation
    * could use analogous data to support, and impute missing values, or fit a parametric distribution supported by theory
    * also consider data smoothing via added zero centered kernels around the data, known as smooth bootstrap 

3. For $\ell = 1, \ldots, L$ realizations, do the following:

    * For $i = \alpha, \ldots, n$ data, do the following:

        * draw a random sample with replacement from the sample set or Monte Carlo simulate from the CDF (if available). 

6. Calculate a realization of the summary statistic of interest from the $n$ samples, e.g., $m^\ell$, $\sigma^2_{\ell}$. Return to 3 for another realization.

7. Compile and summarize the $L$ realizations of the statistic of interest.

#### Bootstrap vs. Theory

Does this work? You can compare bootstrap to theory, e.g., for uncertainty in the mean the analytical solution is known as standard error: 

\begin{equation}
\sigma^2_\overline{x} = \frac{\sigma^2_s}{n}
\end{equation}

Extremely powerful - could calculate uncertainty in any statistic!  e.g. P13, skew etc.
* Would not be possible access general uncertainty in any statistic without bootstrap.

This is a very powerful method.  Let's try it out.

#### Getting Started

Here's the steps to get setup in Python with the GeostatsPy package:

1. Install Anaconda 3 on your machine (https://www.anaconda.com/download/). 

That's all!

#### Load the Required Libraries

We will also need some standard Python packages. These should have been installed with Anaconda 3.

In [1]:
supress_warnings = True                                   # ignore warnings
%matplotlib inline
from ipywidgets import interactive                        # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
import matplotlib.pyplot as plt                           # plotting
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator) # control of axes ticks
plt.rc('axes', axisbelow=True)                            # set axes and grids in the background for all plots
import numpy as np                                        # working with arrays
import pandas as pd                                       # working with DataFrames
import seaborn as sns                                     # for matrix scatter plots
from scipy.stats import triang                            # parametric distributions
from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import uniform
from scipy.stats import triang
from scipy import stats                                   # statistical calculations
import random                                             # random drawing / bootstrap realizations of the data
from matplotlib.gridspec import GridSpec
import warnings
plt.rc('axes', axisbelow=True)                            # grid behind plotting elements
if supress_warnings == True:
    import warnings                                       # supress any warnings for this demonstration
    warnings.filterwarnings('ignore')   

#### Bootstrap Realizations from the Cowbow Hat

It's a lot of work to draw $n$ colored blocks from a cowbow hat with replacement, calculate the realization of the proportion of red and yellow blocks and repeat $L$ times to calculate the uncertainty distribution for the proportion.

* Let's let the compute sample for us! 

This dashboard makes and visualizes bootstrap realizations.

In [2]:
seed = 73073
# parameters for the synthetic dataset
bins = np.linspace(0,1000,1000)

# interactive calculation of the sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                                Simple Boostrap Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

a = widgets.IntSlider(min=0, max = 100, value = 2, step = 1, description = 'n(red)',orientation='horizontal',layout=Layout(width='400px', height='30px'),continuous_update=False)
a.style.handle_color = 'red'

b = widgets.IntSlider(min=0, max = 100, value = 3, step = 1, description = 'n(green)',orientation='horizontal',layout=Layout(width='400px', height='30px'),continuous_update=False)
b.style.handle_color = 'yellow'

c = widgets.IntSlider(min=1, max = 16, value = 3, step = 1, description = 'L',orientation='horizontal',layout=Layout(width='400px', height='30px'),continuous_update=False)
c.style.handle_color = 'black'

ui = widgets.HBox([a,b,c],)                               # basic widget formatting           
ui2 = widgets.VBox([l,ui],)

def f_make(a, b, c):                                      # function to take parameters, make sample and plot
    red_freq = make_data(a, b, c)
    np.random.seed(seed=seed) 
    labels = ['Red', 'Yellow']
    #nrows = int(np.round(np.sqrt(c)+0.4,0)); ncols = int(np.round(c / nrows + 0.4,0))
    nrows = 4; ncols = 4
    plt.clf()
    
    for i in range(0, c): 
        plt.subplot(ncols,nrows,i + 1) 
        draw = [red_freq[i],a + b - red_freq[i]]
        plt.grid(zorder=0, color='black', axis = 'y', alpha = 0.2); plt.ylim(0,a + b); 
        plt.ylabel('Frequency'); plt.xlabel('Blocks Drawn')
        plt.yticks(np.arange(0,a + b + 1,max(1,round((a+b)/10))))
        barlist = plt.bar(labels,draw,edgecolor = "black",linewidth = 1,alpha = 0.8); plt.title('Realization #' + str(i+1),zorder = 1) 
        barlist[0].set_color('red'); barlist[1].set_color('yellow')
        barlist[0].set_edgecolor('black'); barlist[1].set_edgecolor('black')
            
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2 * nrows, wspace=0.2, hspace=0.2)
    plt.show()

def make_data(a, b, c):                                   # function to check parameters and make sample   
    prop_red = np.zeros(c)
    for i in range(0, c): 
        prop_red[i] = np.random.multinomial(a+b,[a/(a+b),b/(a+b)], size = 1)[0][0]
    return prop_red

# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(f_make, {'a': a, 'b': b, 'c': c})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating

### Simple Bootstrap Demonstration - Drawing Red and Green Balls from a Virtual Cowboy Hat

* drawing red and green balls from a hat with replacement to access uncertainty in the proportion 

* interactive plot demonstration with ipywidget, matplotlib packages

#### Michael Pyrcz, Associate Professor, University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1) | [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy)

### The Problem

Let's simulate bootstrap, resampling with replacement from a hat with $n_{red}$ and $n_{green}$ balls

* **$n_{red}$**: number of red balls in the sample (placed in the hat)

* **$n_{green}$**: number of green balls in the sample (placed in the hat)

* **$L$**: number of bootstrap realizations

In [3]:
display(ui2, interactive_plot)                            # display the interactive plot

Output(outputs=({'output_type': 'display_data', 'metadata': {}, 'data': {'text/plain': '<Figure size 640x480 w…

Now let's look at a line chart of proportion of red and green balls over many bootstrap realizations. 

* for this we will use a line chart, as it is difficult to visualize many histograms.

In [4]:
# parameters for the synthetic dataset
bins = np.linspace(0,1000,1000)

# interactive calculation of the sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                                Boostrap from a Cowbow Hat Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

a = widgets.IntSlider(min=0, max = 100, value = 2, step = 1, description = 'n(red)',orientation='horizontal',layout=Layout(width='400px', height='30px'),continuous_update=False)
a.style.handle_color = 'red'

b = widgets.IntSlider(min=0, max = 100, value = 3, step = 1, description = 'n(yellow)',orientation='horizontal',layout=Layout(width='400px', height='30px'),continuous_update=False)
b.style.handle_color = 'yellow'

c = widgets.IntSlider(min=1, max = 1000, value = 20, step = 1, description = 'L',orientation='horizontal',layout=Layout(width='400px', height='30px'),continuous_update=False)
c.style.handle_color = 'gray'

ui = widgets.HBox([a,b,c],)                               # basic widget formatting           
ui3 = widgets.VBox([l,ui],)

def f_make3(a, b, c):                                      # function to take parameters, make sample and plot
    np.random.seed(seed=seed)
    start_CI = 3; Plow = 30; Phigh = 70
    red_freq = make_data(a, b, c) 
    green_freq = (a + b) - red_freq
    prop_red = red_freq / (a + b)
    prop_green = green_freq / (a + b)
    labels = ['Red', 'Yellow']
    trials = np.arange(1,c+1,1)

    prop_red_cumul = np.cumsum(prop_red, axis=0, dtype=None, out=None)/np.arange(1,len(prop_red)+1,1)
    prop_green_cumul = 1.0 - prop_red_cumul
    
    cumul_per_trial = np.arange(start_CI,c+1,1)
    Plow_red = []; Phigh_red = []; Plow_green = []; Phigh_green = [];
    for l in range(start_CI,len(prop_red)+1):
        Plow_red.append(np.percentile(prop_red[:l],Plow))
        Phigh_red.append(np.percentile(prop_red[:l],Phigh))
        Plow_green.append(np.percentile(prop_green[:l],Plow)) 
        Phigh_green.append(np.percentile(prop_green[:l],Phigh)) 
    
    plt.subplot(1,2,1) 
    plt.plot([1,max(c,2)],[a/(a+b),a/(a+b)],color='red',alpha=1.0,lw=2,zorder=3)
    plt.plot([1,max(c,2)],[a/(a+b),a/(a+b)],color='black',alpha=0.6,lw=4,zorder=1)
    plt.annotate(r'$prop_{red}$',[(max(c,2)-1)*0.02+1,a/(a+b)+0.02],zorder=1001)
    
    plt.plot([1,max(c,2)],[b/(a+b),b/(a+b)],color='yellow',alpha=1.0,lw=2,zorder=3)
    plt.plot([1,max(c,2)],[b/(a+b),b/(a+b)],color='black',alpha=0.6,lw=4,zorder=1)
    plt.annotate(r'$prop_{yellow}$',[(max(c,2)-1)*0.02+1,b/(a+b)+0.02],zorder=1001)
    
    plt.plot(trials,prop_red,color='red',alpha=0.6,lw=1,ls='--',zorder=10)
    plt.scatter(trials,prop_red,color='red',edgecolor='black',alpha=1.0,s=20,label=r'$prop_{red}^{\ell}$',zorder=100)
    plt.plot(trials,prop_green,color='yellow',alpha=1.0,lw=1,ls='--',zorder=10)
    plt.scatter(trials,prop_green,color='yellow',edgecolor='black',alpha=1.0,s=20,label=r'$prop_{yellow}^{\ell}$',zorder=100)

    plt.xlim([1,max(c,2)]); plt.ylim([0,1])
    plt.xlabel('Bootstrap Realizations, $L$'); plt.ylabel('Proportion'); plt.title('Bootstrap Realizations of Proportions of Blocks')
    plt.legend(loc='upper right')
    if c < 30:
        plt.gca().set_xticks(np.arange(1,c+1,1.0))
    plt.gca().set_yticks(np.arange(0,1.1,0.1))
    plt.gca().grid(True, which='major',axis='y',linewidth = 1.0); plt.gca().grid(True, which='minor',axis='y',linewidth = 0.2) # add y grids
    plt.gca().tick_params(which='major',axis='y'); plt.gca().tick_params(which='minor',axis='y',length=4)
    plt.gca().yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks    
    #plt.gca().xaxis.set_minor_locator(AutoMinorLocator()); 
    plt.gca().grid(True, which='major',axis='y',linewidth = 1.0);
    
    
    plt.subplot(1,2,2) 
    plt.plot([1,max(c,2)],[a/(a+b),a/(a+b)],color='red',alpha=1.0,lw=2,zorder=3)
    plt.plot([1,max(c,2)],[a/(a+b),a/(a+b)],color='black',alpha=0.6,lw=4,zorder=1)
    plt.annotate(r'$prop_{red}$',[(max(c,2)-1)*0.02+1,a/(a+b)+0.02],zorder=1001)
    
    plt.plot([1,max(c,2)],[b/(a+b),b/(a+b)],color='yellow',alpha=1.0,lw=2,zorder=3)
    plt.plot([1,max(c,2)],[b/(a+b),b/(a+b)],color='black',alpha=0.6,lw=4,zorder=1)
    plt.annotate(r'$prop_{yellow}$',[(max(c,2)-1)*0.02+1,b/(a+b)+0.02],zorder=1001)    

    plt.plot(trials,prop_red_cumul,color='red',alpha=0.6,lw=1,ls='--',zorder=10)
    plt.scatter(trials,prop_red_cumul,color='red',edgecolor='black',alpha=1.0,s=20,label=r'$Prop_{red}^{\leq \ell}$',zorder=100)
    plt.plot(trials,prop_green_cumul,color='yellow',alpha=1.0,lw=1,ls='--',zorder=10)
    plt.scatter(trials,prop_green_cumul,color='yellow',edgecolor='black',alpha=1.0,s=20,label=r'$Prop_{yellow}^{\leq \ell}$',zorder=100)
    
    plt.fill_between(cumul_per_trial,Phigh_red,Plow_red,color='red',edgecolor=None,alpha=0.15,zorder=1)
    plt.plot(cumul_per_trial,Plow_red,color='red',alpha=0.3,lw=1,zorder=2) 
    plt.plot(cumul_per_trial,Phigh_red,color='red',alpha=0.3,lw=1,zorder=2) 
    
    plt.fill_between(cumul_per_trial,Phigh_green,Plow_green,color='yellow',edgecolor=None,alpha=0.15,zorder=1)
    plt.plot(cumul_per_trial,Plow_green,color='yellow',alpha=0.3,lw=1,zorder=2) 
    plt.plot(cumul_per_trial,Phigh_green,color='yellow',alpha=0.3,lw=1,zorder=2) 
    
    plt.xlim([1,max(c,2)]); plt.ylim([0,1])
    plt.xlabel('Bootstrap Realizations, $L$'); plt.ylabel('Proportion'); plt.title('Cumulative Statistics Over Bootstrap Realizations')
    plt.legend(loc='upper right')
    if c < 30:
        plt.gca().set_xticks(np.arange(1,c+1,1.0))
    plt.gca().set_yticks(np.arange(0,1.1,0.1))
    plt.gca().grid(True, which='major',axis='y',linewidth = 1.0); plt.gca().grid(True, which='minor',axis='y',linewidth = 0.2) # add y grids
    plt.gca().tick_params(which='major',axis='y'); plt.gca().tick_params(which='minor',axis='y',length=4)
    plt.gca().yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks    
    plt.gca().grid(True, which='major',axis='y',linewidth = 1.0);
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.2, hspace=0.2)
    plt.show()

def make_data(a, b, c):                                   # function to check parameters and make sample   
    prop_red = np.zeros(c)
    for i in range(0, c): 
        prop_red[i] = np.random.multinomial(a+b,[a/(a+b),b/(a+b)], size = 1)[0][0]
    return prop_red

# connect the function to make the samples and plot to the widgets    
interactive_plot3 = widgets.interactive_output(f_make3, {'a': a, 'b': b, 'c': c})
interactive_plot3.clear_output(wait = True)                # reduce flickering by delaying plot updating

### Simple Bootstrap Demonstration with a Cowbow Hat - Uncertainty in Proportions

Let's look at the individual bootstrap realizations and cumulative uncertainty model over those realizations.

#### Michael Pyrcz, Professor, The University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1) | [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy)

### The Problem

Let's simulate bootstrap, resampling with replacement from a hat with $n_{red}$ and $n_{yellow}$ blocks

* **$n_{red}$**: number of red balls in the sample (placed in the hat)

* **$n_{yellow}$**: number of green balls in the sample (placed in the hat)

* **$L$**: number of bootstrap realizations

In [5]:
display(ui3, interactive_plot3)                            # display the interactive plot

Output(outputs=({'output_type': 'display_data', 'metadata': {}, 'data': {'text/plain': '<Figure size 640x480 w…

#### Summarizing Bootstrap Uncertainty

* Run more bootstrap realizations and evaluate the uncertainty model

Now instead of looking at each bootstrap result, let's look at all realizations and summarize with:

* **box and whisker plot** of the red and green ball frequencies

* **histograms** of the red and green ball frequencies.

In [6]:
# parameters for the synthetic dataset
bins = np.linspace(0,1000,1000)

# interactive calculation of the sample set (control of source parametric distribution and number of samples)
l2 = widgets.Text(value='            Bootstrap Demonstration with a Cowbow Hat, Uncertainty in Proportion of Blocks, Michael Pyrcz, Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

a2 = widgets.IntSlider(min=0, max = 100, value = 2, step = 1, description = 'n(red)',orientation='horizontal',layout=Layout(width='400px', height='20px'),continuous_update=False)
a2.style.handle_color = 'red'

b2 = widgets.IntSlider(min=0, max = 100, value = 3, step = 1, description = 'n(yellow)',orientation='horizontal',layout=Layout(width='400px', height='20px'),continuous_update=False)
b2.style.handle_color = 'yellow'

c2 = widgets.IntSlider(min=1, max = 1000, value = 30, step = 1, description = 'L',orientation='horizontal',layout=Layout(width='400px', height='20px'),continuous_update=False)
c2.style.handle_color = 'gray'

uib = widgets.HBox([a2,b2,c2],)                               # basic widget formatting           
uib2 = widgets.VBox([l2,uib],)

def s_make(a, b, c):                                      # function to take parameters, make sample and plot
    np.random.seed(seed=seed)
    red_freq = make_data(a, b, c)
    green_freq = (a + b) - red_freq
    prop_red = red_freq / (a + b)
    prop_yellow = green_freq / (a + b)
    labels = ['Red Blocks', 'Yellow Blocks']
    bins = np.linspace(0,1.0,100)

    fig = plt.figure(constrained_layout=False)
    gs = GridSpec(2, 2, figure=fig)
    
    ax1 = fig.add_subplot(gs[:, 0])
    boxplot = ax1.boxplot([prop_red,prop_yellow],labels = labels, notch = True, sym = '+',patch_artist=True) 
    ax1.set_ylabel('Proportion of Blocks'); ax1.set_xlabel('Block Color');ax1.set_title('Bootstrap Uncertainty - Proportion Distributions')
    ax1.set_yticks(np.arange(0,1.1,0.1)); ax1.set_ylim([0,1])
    ax1.grid(True, which='major',axis='y',linewidth = 1.0); ax1.grid(True, which='minor',axis='y',linewidth = 0.2) # add y grids
    ax1.tick_params(which='major',length=7); ax1.tick_params(which='minor', length=4)
    ax1.xaxis.set_minor_locator(AutoMinorLocator()); ax1.yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks
    
    colors = ['red','yellow']
    for patch, color in zip(boxplot['boxes'], colors):
        patch.set_facecolor(color)
    for patch, color in zip(boxplot['medians'], colors):
        patch.set_color('black')
        
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.hist(prop_red,alpha=0.8,color="red",edgecolor="black", bins = bins)
    ax2.set_title('Red Block Bootstrap Uncertainty - Proportion Distributions'); ax2.set_xlabel('Proportions of Red Blocks'); ax2.set_ylabel('Frequency') 
    ax2.grid(True, which='major',linewidth = 1.0); ax2.grid(True, which='minor',linewidth = 0.2) # add y grids
    ax2.tick_params(which='major',length=7); ax2.tick_params(which='minor', length=4)
    ax2.xaxis.set_minor_locator(AutoMinorLocator()); ax2.yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks
    ax2.set_xlim([0,1])
    
    ax3 = fig.add_subplot(gs[1, 1])
    ax3.hist(prop_yellow,alpha=0.8,color="yellow",edgecolor="black", bins = bins)
    ax3.set_title('Yellow Block Bootstrap Uncertainty - Proportion Distribution'); ax3.set_xlabel('Proportion of Yellow Blocks'); ax3.set_ylabel('Frequency') 
    ax3.grid(True, which='major',linewidth = 1.0); ax3.grid(True, which='minor',linewidth = 0.2) # add y grids
    ax3.tick_params(which='major',length=7); ax3.tick_params(which='minor', length=4)
    ax3.xaxis.set_minor_locator(AutoMinorLocator()); ax3.yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks
    ax3.set_xlim([0,1])
    
#     print('Uncertainty in Proportion of Red Balls: ')
#     print('P10: ' + str(np.round(np.percentile(prop_red,10),2)) + ', P50: ' + str(np.round(np.percentile(prop_red,50),2)) + ', P90: ' + str(np.round(np.percentile(prop_red,90),2)))

    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=2.1, wspace=0.2, hspace=0.3)
    plt.show()

# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(s_make, {'a': a2, 'b': b2, 'c': c2})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating

### Simple Bootstrap Demonstration with a Cowbow Hat - Uncertainty in Proportions

Let's look at bootstrap sample distributions over all realizations.

#### Michael Pyrcz, Professor, The University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1) | [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy)

### The Problem

Let's simulate bootstrap, resampling with replacement from a hat with $n_{red}$ and $n_{yellow}$ blocks

* **$n_{red}$**: number of red balls in the sample (placed in the hat)

* **$n_{yellow}$**: number of green balls in the sample (placed in the hat)

* **$L$**: number of bootstrap realizations

In [7]:
display(uib2, interactive_plot)                           # display the interactive plot

Output(outputs=({'output_type': 'display_data', 'metadata': {}, 'data': {'text/plain': '<Figure size 640x480 w…

#### Observations

Some observations:

* sampling distribution for proportions become discrete with too few samples, only $n$ cases are possible

* enough bootstrap realizations are rquired for stable statistics


#### Comments

This was a simple demonstration of interactive plots in Jupyter Notebook Python with the ipywidgets and matplotlib packages. 

I have many other demonstrations on data analytics and machine learning, e.g. on the basics of working with DataFrames, ndarrays, univariate statistics, plotting data, declustering, data transformations, trend modeling and many other workflows available at https://github.com/GeostatsGuy/PythonNumericalDemos and https://github.com/GeostatsGuy/GeostatsPy. 
  
I hope this was helpful,

*Michael*

#### The Author:

### Michael Pyrcz, Professor, The University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Professor, Cockrell School of Engineering and The Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)
